In [1]:
import yfinance as yf
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression

In [2]:
univ = [
    # --- TECHNOLOGY ---
    'MSFT', 'AAPL', 'INTC', 'CSCO', 'ORCL', 'IBM', 'ADBE', 'TXN', 'NVDA', 'QCOM',
    'AMAT', 'ADI', 'MU', 'LRCX', 'KLAC', 'AMD', 'APH', 'GLW', 'HPQ', 'MSI',
    'ADP', 'PAYX', 'FISV', 'FIS', 'CTSH', 'INTU', 'ADSK', 'SNPS', 'CDNS', 'MCHP',
    'STM', 'UMC', 'TSM', 'ASX', 'TEL', 'TER', 'NTAP', 'STX', 'WDC', 'ZBRA',
    'TRMB', 'TYL', 'AKAM', 'VRSN',

    # --- HEALTHCARE ---
    'JNJ', 'PFE', 'MRK', 'LLY', 'UNH', 'ABT', 'BMY', 'AMGN', 'GILD', 'BIIB',
    'SYK', 'MDT', 'BAX', 'BDX', 'CVS', 'CI', 'HUM', 'MCK', 'CAH', 'LH',
    'TMO', 'DHR', 'ISRG', 'EW', 'BSX', 'ZBH', 'STE', 'COO', 'HOLX', 'XRAY',
    'DGX', 'CNC', 'MOH', 'A', 'MTD', 'WAT', 'TECH', 'BIO',
    'RGEN', 'VRTX', 'REGN', 'INCY',

    # --- FINANCIALS ---
    'JPM', 'BAC', 'WFC', 'C', 'AXP', 'GS', 'MS', 'USB', 'BK', 'STT',
    'PGR', 'ALL', 'AIG', 'HIG', 'TRV', 'MMC', 'AON', 'BEN', 'SCHW', 'MCO',
    'SPGI', 'PNC', 'TFC', 'KEY', 'FITB', 'MTB', 'HBAN', 'RF', 'CMA', 'ZION',
    'L', 'CINF', 'WRB', 'AFL', 'PRU', 'MET', 'PFG', 'LNC', 'UNM',
    'RJF', 'SEIC', 'TROW', 'IVZ', 'AMG', 'BLK', 'ICE', 'CME', 'NDAQ', 'JKHY',
    'BRO', 'AJG', 'WTM', 'FAF',

    # --- CONSUMER STAPLES ---
    'KO', 'PEP', 'PG', 'WMT', 'COST', 'CL', 'MO', 'SYY', 'K', 'GIS',
    'HSY', 'CLX', 'MKC', 'TSN', 'CAG', 'TAP', 'EL', 'ADM', 'HRL', 'SJM',
    'KMB', 'CPB', 'SBUX', 'KR', 'DG', 'DLTR', 'TGT',
    'CHD', 'STZ', 'BF-B', 'NWL',

    # --- CONSUMER DISCRETIONARY ---
    'HD', 'LOW', 'MCD', 'NKE', 'F', 'DIS', 'TJX', 'VFC', 'YUM', 'DRI',
    'CMG', 'MAR', 'HLT', 'CCL', 'RCL', 'HAS', 'MAT', 'BBY', 'GPC',
    'AZO', 'ORLY', 'KMX', 'ROST', 'LB', 'M', 'KSS', 'DDS',
    'LEG', 'MHK', 'WHR', 'LEN', 'PHM',

    # --- INDUSTRIALS ---
    'GE', 'BA', 'CAT', 'HON', 'LMT', 'RTX', 'GD', 'MMM', 'UNP', 'FDX',
    'UPS', 'DE', 'EMR', 'ITW', 'ETN', 'PH', 'DOV', 'CMI', 'PCAR', 'NSC',
    'CSX', 'GWW', 'FAST', 'VMI', 'RSG', 'WM', 'CTAS', 'GPN', 'EFX', 'JCI',
    'TXT', 'NOC', 'LHX', 'HII', 'TDG', 'AME', 'ROK', 'SWK', 'SNA', 'MAS',

    # --- ENERGY & MATERIALS ---
    'XOM', 'CVX', 'COP', 'SLB', 'HAL', 'VLO', 'OXY', 'DVN',
    'EOG', 'APA', 'BKR', 'NEM', 'FCX', 'APD', 'ECL', 'SHW', 'PPG',
    'LYB', 'DOW', 'DD', 'IP', 'NUE',

    # --- UTILITIES ---
    'NEE', 'DUK', 'SO', 'AEP', 'ED', 'PEG', 'XEL', 'EIX', 'ETR', 'D',
    'WEC', 'ES', 'AWK', 'SRE', 'FE', 'CMS', 'DTE', 'PPL', 'CNP', 'NI',

    # --- REAL ESTATE ---
    'PLD', 'SPG', 'PSA', 'O', 'VTR', 'BXP', 'AVB', 'EQR', 'ESS',
    'MAA', 'UDR', 'HST', 'VNO', 'SLG'
]

In [3]:
price_volume_data = pd.read_pickle('price_volume_data.pkl')
rets = price_volume_data['Close'].pct_change(fill_method=None).dropna(how='all')

In [4]:
window_PCA = 252
window_OU = 60
start_trading_date = '2000-01-05'
num_factors = 5

In [5]:
def PCA_process(rets_window, num_factors):
    # Construct empirical correlation matrix
    corr_matrix = rets_window.corr()

    # Eigen decomposition
    eigenvalues, eigenvectors = np.linalg.eigh(corr_matrix)

    # Sort eigenvalues and eigenvectors in descending order
    eigenvalues = eigenvalues[::-1]
    eigenvectors = eigenvectors[:, ::-1]
    eigenvectors = pd.DataFrame(eigenvectors,
                                index=rets_window.columns,
                                columns=np.arange(1, len(eigenvalues) + 1))

    # Calculate percentage of variance explained by each factor
    variance_pct = eigenvalues / np.sum(eigenvalues)

    # For now focusing on 15 Factor model so num_factors=15, but can be adjusted for variable number by reaching a threshold of variance explained

    # Calculate weights & factor returns for eigenportfolio
    std = rets_window.std()
    eigenvectors_selected = eigenvectors.loc[:, 1:num_factors]
    eigen_weights = eigenvectors_selected.div(std, axis=0)

    return eigen_weights, variance_pct

In [6]:
def OU_process(eigenportfolio_rets, rets_window, window_OU):
    # Vectorized OLS for residual calculation
    F = eigenportfolio_rets.tail(window_OU)
    F.insert(0, 0, 1)
    R = rets_window.tail(window_OU)

    # Calculate the beta coefficients (weight of each factor)
    beta = np.linalg.inv(F.T @ F) @ F.T @ R
    beta = pd.DataFrame(beta.values,
                        index=F.columns,
                        columns=R.columns)
    residuals = R - (F @ beta)
    cum_residuals = residuals.cumsum()

    # Calculate OU parameters (1-lag regression model)
    X = cum_residuals.iloc[:-1, :]
    Y = cum_residuals.iloc[1:, :]

    # Univariate approach to calculate OU parameters
    numerator = ((X - X.mean(axis=0)).values * (Y - Y.mean(axis=0)).values).sum(axis=0)
    denominator = ((X - X.mean(axis=0))**2).sum(axis=0)

    # denominator[denominator == 0] = np.nan

    b = numerator / denominator
    mask = (b >= 1) | (b <= 0)
    b[mask] = 0.1
    a = Y.mean(axis=0) - (b * X.mean(axis=0))

    zeta = Y.values - (a.values + b.values * X.values)
    zeta = pd.DataFrame(zeta,
                        index=Y.index,
                        columns=Y.columns)

    m = a / (1 - b)
    centered_m = m - m.mean()
    kappa = -np.log(b) * 252
    kappa[mask] = 0

    sigma_eq = zeta.std(ddof=1) / np.sqrt(1 - b**2)
    s_score = -centered_m / sigma_eq
    s_score[mask] = 0

    return s_score, kappa, sigma_eq


In [7]:
def signal_gen():
    days = 0
    Dates = rets.index[rets.index >= start_trading_date]
    S_score = pd.DataFrame(index=Dates, columns=rets.columns)
    Kappa = pd.DataFrame(index=Dates, columns=rets.columns)
    Sigma_eq = pd.DataFrame(index=Dates, columns=rets.columns)

    for date in Dates:
        # Filter returns for the estimation window
        current_idx = rets.index.get_loc(date)
        rets_window = rets.iloc[current_idx - window_PCA:current_idx]

        if days % 30 == 0:
            # Filter NaN values
            nan_count = rets_window.isna().sum()
            mask = nan_count == 0
            valid_columns = rets_window.columns[mask]
            rets_window = rets_window[valid_columns].fillna(0)

            eigen_weights, variance_pct = PCA_process(rets_window=rets_window,
                                                    num_factors=num_factors)
        else:
            rets_window = rets_window[valid_columns].fillna(0)

        eigenportfolio_rets = rets_window @ eigen_weights
        s_score, kappa, sigma_eq = OU_process(eigenportfolio_rets, rets_window, window_OU)

        S_score.loc[date, valid_columns] = s_score
        Kappa.loc[date, valid_columns] = kappa
        Sigma_eq.loc[date, valid_columns] = sigma_eq

        days += 1
        if days % 30 == 0:
            print(f'Processed {days} days')
        elif date == Dates[-1]:
            print('Processing complete.')
    
    return S_score, Kappa, Sigma_eq

S_score, Kappa, Sigma_eq = signal_gen()

LinAlgError: Singular matrix

In [ ]:
def gen_positions(S_score, Kappa, Sigma_eq):
    days = 0
    Dates = S_score.index
    current_state = pd.Series(0,
                                index=S_score.columns)
    OU_weight = pd.Series(0,
                                index=S_score.columns)
    state_history = pd.DataFrame(index=S_score.index,
                                    columns=S_score.columns)
    weight_history = pd.DataFrame(index=S_score.index,
                                  columns=S_score.columns)

    # Initialize s_score signals
    ENTRY_LONG = -1.50
    ENTRY_SHORT = 1.50
    EXIT_SHORT = 0.25
    EXIT_LONG = -0.25
    STOP_LOSS = 2.50

    for date in Dates:
        s_score = S_score.loc[date]
        kappa = Kappa.loc[date]
        sigma_eq = Sigma_eq.loc[date]

        # Exit positions
        exit_long = (current_state == 1) & ((s_score > EXIT_LONG) | (s_score < STOP_LOSS))
        current_state[exit_long] = 0
        exit_short = (current_state == -1) & ((s_score < EXIT_SHORT) | (s_score > -STOP_LOSS))
        current_state[exit_short] = 0

        # Filter by kappa > 8.4 for quick reversion times
        mask = kappa > 10

        # Open positions
        enter_long = (current_state == 0) & (s_score < ENTRY_LONG) & mask
        current_state[enter_long] = 1
        enter_short = (current_state == 0) & (s_score > ENTRY_SHORT) & mask
        current_state[enter_short] = -1

        # Update state history
        state_history.loc[date] = current_state

        # # Calculate OU weights
        # OU_weights = s_score.abs() / sigma_eq
        # active_weights = OU_weights * current_state.abs()
        # weight_history.loc[date] = active_weights

        days += 1
        if days % 30 == 0:
            print(f'Processed {days} days')
        elif date == Dates[-1]:
            print(f'Processed {days} days')

    print('Processing complete.')

    return state_history #, weight_history

state_history = gen_positions(S_score, Kappa, Sigma_eq)

# IMPLEMENT BETA HEDGING BY BETTING ON FACTORS AGAINST PORTFOLIO
# EXPERIMENT HOW VALUE OF KAPPA AFFECTS PERFORMANCE METRICS


In [ ]:
def calc_PnL(state_history, rets):
    # Initialize capital
    INITIAL_CAPITAL = 1000
    PCT_PER_TRADE = 0.01

    # Calculate raw weights
    states = state_history.shift(1).fillna(0)
    # weights = weight_history.shift(1).fillna(0)
    # raw_weights = states * weights

    # Normalize weights
    # portfolio_weights = raw_weights.div(raw_weights.abs().sum(axis=1), axis=0).fillna(0)
    # portfolio_weights = portfolio_weights * PCT_PER_TRADE
    portfolio_weights = states * PCT_PER_TRADE

    # Calculate dollar returns
    rets = rets.loc[states.index]
    daily_strat_rets = (portfolio_weights * rets).sum(axis=1)

    # Calculate compounded equity curve
    PnL_curve = (1 + daily_strat_rets).cumprod() * INITIAL_CAPITAL

    return PnL_curve, portfolio_weights, daily_strat_rets

PnL_curve, portfolio_weights, daily_strat_rets = calc_PnL(state_history, rets)

PnL_curve.plot(figsize=(12, 6), title='P&L Curve')
plt.show()


# NEED TO IMPLEMENT PORTFOLIO WEIGHTS
# Currently fixed amount per trade without using OU params

In [ ]:
annualized_sharpe = daily_strat_rets.mean() / daily_strat_rets.std() * np.sqrt(252)
print(annualized_sharpe)

In [ ]:
Kappa['MSFT'].plot(figsize=(12, 6), title='Kappa of MSFT')

In [ ]:
S_score['MSFT'].plot(figsize=(12, 6), title='S-score of MSFT')